blublahblah123

### Extracting Files

Install gpg.exe file to decrypt the file.

import tarfile

tar = tarfile.open("aila20-task1.tar.gz", "r:gz")

tar.extractall(r'C:\Users\Aditi\AILA')

#### This is how I am planning to proceed
1. Data Cleaning
2. To find the relavant case docs
    2a. Deep Bayesian Learning
        Method : Generative Model
           1a. Document: Distribution over a topic
           1b. Topic : Distribution over a word
    2b. Bayesian Deep Learning
        Method : Uncertinity Model
            2a. Aleatoric uncertinity
            2b. Epistemic uncertinity
3. References
https://medium.com/@adriensieg/text-similarities-da019229c894
******

### About the Dataset

ProblemLink: https://sites.google.com/view/aila-2020/dataset-evaluation-plan?authuser=0

Google Group For Quiries: https://groups.google.com/forum/?utm_medium=email&utm_source=footer#!topic/aila-fire/EwDlG4cRwyY


The zip file consists of the following documents:

1. Query_doc.txt -- contains the 50 queries, i.e., description of legal situations
2. Object_casedocs -- contains 2,914 prior case documents, some of which are relevant to the queries (to be used for Task 1).
3. Object_statutes -- contains the title and textual description of 197 statutes (to be used for Task 2)
4. Gold standard annotations for relevant precedents and
5. README.txt file that specifies the formats of the other files / folders

### Task 1A : Identifying relevant prior cases

In [1]:
import pandas as pd
import numpy as np

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

import gensim #Gensim is billed as a Natural Language Processing package that does ‘Topic Modeling for Humans’
from gensim import corpora, models, similarities

import glob
import os

import jieba

In [2]:
with open ('C:/Users/Aditi/AILA/dataset/Query_doc.txt', 'r') as f:
    all_lines = f.read().splitlines() 
    
print("Total queries in the dataset: ", len(all_lines)) 

Total queries in the dataset:  50


In [3]:
path = 'C:/Users/Aditi/AILA/dataset/Object_casedocs/'
count = 0
for filename in glob.glob(os.path.join(path, '*.txt')):
    count = count + 1
print("Total casedocs in the train dataset: ", count)

Total casedocs in the train dataset:  2914


In [15]:
path = 'C:/Users/Aditi/AILA/dataset/Object_casedocs/'

df = pd.DataFrame()
j = 0 ### iterate for a column

for filename in glob.glob(os.path.join(path, '*.txt')):
    with open (filename, 'r') as f:
        sentenses = [] ### Store all the sentences into a list
        tokens = sent_tokenize(f.read())
        
        for line in tokens:
            sentenses.append(line)
    
        indices = [i for i, s in enumerate(sentenses) if 'The Judg' in s]
    
        df.loc[j,'Basic Info'] = sentenses[0:indices[0]+1]
        df.loc[j,'Case Details' ] = sentenses[indices[0]+1:len(sentenses)]
        
        j = j + 1

['Masud Khan v State Of Uttar Pradesh\nSupreme Court of India\n\n26 September 1973\nWrit Petition No.', '117 of 1973\nThe Judgment was delivered by : A. Alagiriswami, J.']
['Prabhakaran Nair, Etc.', 'v State Of Tamil Nadu And Ors.', 'Supreme Court of India\n\n3 September 1987\nWrit Petition No.', '506 of 1986\nThe Judgment was delivered by: Sabyasachi Mukharji, J.']
['Hiten P. Dalal v Bratindranath Banerjee\nSupreme Court of India\n\n11 July 2001\nAppeal (Cr.)', '688 of 1995\nThe Judgment was delivered by : Ruma Pal, J\n1.']
['Ashok Kumar and Others v State of Tamil Nadu\nSupreme Court of India\n\n5  May  2006\nAppeal (Crl.)', '1533 of 2004\nThe Judgment was delivered by : S. B. Sinha, J.']


IndexError: list index out of range

In [16]:
with open ('C:/Users/Aditi/AILA/dataset/Object_casedocs/C1.txt', 'r') as f:
    sentenses = [] ### Store all the sentences into a list
    tokens = sent_tokenize(f.read())
        
    for line in tokens:
        sentenses.append(line)
    
    indices = [i for i, s in enumerate(sentenses) if 'The Judg' in s]
    
    df.loc[j,'Basic Info'] = sentenses[0:indices[0]+1]

ValueError: Must have equal len keys and value when setting with an iterable

In [134]:
gen_docs = [[w.lower() for w in word_tokenize(text)] for text in file_docs]

In [135]:
dictionary = gensim.corpora.Dictionary(gen_docs)

In [136]:
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]

In [137]:
tf_idf = gensim.models.TfidfModel(corpus)

In [138]:
sims = gensim.similarities.Similarity('C:/Users/Aditi/Similarity/workdir',tf_idf[corpus],num_features=len(dictionary))

In [144]:

    avg_sims = [] # array of averages
    file2_docs = []
    with open (filename, 'r') as f:
        tokens = sent_tokenize(f.read())
        for line in tokens:
            file2_docs.append(line)

    # for line in query documents
    for line in file2_docs:
        # tokenize words
        query_doc = [w.lower() for w in word_tokenize(line)]
        # create bag of words
        query_doc_bow = dictionary.doc2bow(query_doc)
        # find similarity for each document
        query_doc_tf_idf = tf_idf[query_doc_bow]
        # calculate sum of similarities for each query doc
        sum_of_sims =(np.sum(sims[query_doc_tf_idf], dtype=np.float32))
        # calculate average of similarity for each query doc
        avg = sum_of_sims / len(file_docs)
        # add average values into array
        avg_sims.append(avg)  
    # calculate total average
    total_avg = np.sum(avg_sims, dtype=np.float)
    if total_avg >= 50:
        print(filename, percentage_of_similarity)

C:/Users/Aditi/AILA/dataset/Object_casedocs\C101.txt 190
C:/Users/Aditi/AILA/dataset/Object_casedocs\C1149.txt 190
C:/Users/Aditi/AILA/dataset/Object_casedocs\C14.txt 190
C:/Users/Aditi/AILA/dataset/Object_casedocs\C1512.txt 190
C:/Users/Aditi/AILA/dataset/Object_casedocs\C169.txt 190
C:/Users/Aditi/AILA/dataset/Object_casedocs\C1714.txt 190
C:/Users/Aditi/AILA/dataset/Object_casedocs\C1777.txt 190
C:/Users/Aditi/AILA/dataset/Object_casedocs\C2356.txt 190


In [127]:
# calculate total average
total_avg = np.sum(avg_sims, dtype=np.float)
# round the value and multiply by 100 to format it as percentage
percentage_of_similarity = round(float(total_avg) * 100)
# if percentage is greater than 100
# that means documents are almost same
if percentage_of_similarity >= 100:
    percentage_of_similarity = 100

In [128]:
total_avg

3.478158907682607

Data Cleaning(Remove names other important stuff)

Measure of similarity can be qualitative and/or quantitative. In qualitative, the assessment is done against 
subjective criteria such as theme, sentiment, overall meaning, etc. 
In the quantitative, numerical parameters such as length of the document, number of keywords, common words, etc. are compared. 

In [ ]:
Deep Bayesian Learning

Bayesian Deep Learning ( Tensorflow Probablilty)
aleatoric and epistemic uncertiny

Entitie
Topics

Dirichlet algorithm

Generative Model
Document: Distribution over a topic
Topic : Distribution over a word